In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from datasets import load_dataset
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from transformers import Trainer, TrainingArguments

In [ ]:
# Load the training and validation sets
train_data = load_dataset("json", data_files={"train": "/content/drive/MyDrive/NQuAD_train.json"})
eval_data = load_dataset("json", data_files={"test": "/content/drive/MyDrive/NQuAD_test.json"})

train_dataset = train_data["train"]
eval_dataset = eval_data["test"]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
print(f"Size of the training set: {len(train_dataset)}")
print(f"Size of the testing set: {len(eval_dataset)}")

Size of the training set: 57750
Size of the testing set: 14438


In [ ]:
# mT5 tokenizer and model
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# data preprocessing
def preprocess_function(examples):
    inputs = [
        f'''
        Choose a correct answer to the following questions.
        Context: {c}
        Question: {q}
        Options: {opts}
        Answer: '''
        for c, q, opts in zip(
            examples["sentences_containing_the_numeral_in_answer_options"],
            examples["question_stem"],
            examples["answer_options"]
        )
    ]
    targets = [str(ans) for ans in examples["target_num"]]

    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding=True)
    labels = tokenizer(text_target=targets, max_length=2, truncation=True, padding=True)

    model_inputs["labels"] = np.array(labels["input_ids"], dtype=np.int64)
    return model_inputs

In [ ]:
import numpy as np

# Input Example
example_data = {
    "sentences_containing_the_numeral_in_answer_options": [
        "隨著iPhone 5、三星Galaxy S4(見附圖)等高階智慧型手機銷售不如預期，也讓市場轉而看好中、低階機種的成長力道，並對晶圓代工40奈米製程的需求轉趨樂觀。"
    ],
    "question_stem": [
        "___nm需求趨緊俏，大摩大升聯電/中芯目標價"
    ],
    "answer_options": [
        [22, 28, 30, 40]
    ],
    "target_num": [
        40
    ]
}

processed_data = preprocess_function(example_data)

print("Input Example:", tokenizer.decode(processed_data["input_ids"][0]))
print("Target Example:", tokenizer.decode(processed_data["labels"][0]))


Input Example: Choose a correct answer to the following questions. Context: 隨著iPhone 5、三星Galaxy S4(見附圖)等高階智慧型手機銷售不如預期,也讓市場轉而看好中、低階機種的成長力道,並對晶圓代工40奈米製程的需求轉趨樂觀。 Question: ___nm需求趨緊俏,大摩大升聯電/中芯目標價 Options: [22, 28, 30, 40] Answer:</s>
Target Example: 40</s>


In [ ]:
# Preprocessing
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Remove other columns
columns_to_remove = ["news_article", "question_stem", "answer_options", "ans", "target_num", "sentences_containing_the_numeral_in_answer_options"]
train_dataset = train_dataset.remove_columns(columns_to_remove)
eval_dataset = eval_dataset.remove_columns(columns_to_remove)

# Convert to PyTorch format
train_dataset.set_format("torch")
eval_dataset.set_format("torch")

Map:   0%|          | 0/57750 [00:00<?, ? examples/s]

Map:   0%|          | 0/14438 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, EarlyStoppingCallback

# Clear GPU memory
torch.cuda.empty_cache()

# Define the data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    label_pad_token_id=-100
)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    run_name="mt5-finetuning-run1",
    report_to="none",
    eval_strategy="steps",
    eval_steps=50,
    logging_steps=100,
    logging_dir="./logs",
    logging_first_step=True,
    logging_nan_inf_filter=False,
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    gradient_accumulation_steps=4,
    predict_with_generate=False,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Add early stopping
)

# Start training
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
50,189.168600,10.942528
100,64.746400,2.299940
150,64.746400,1.583583
200,7.652600,0.997605
250,7.652600,0.867433
300,3.509600,0.706159
350,3.509600,0.667791
400,2.891800,0.654132
450,2.891800,0.603309
500,2.565800,0.524473


Could not locate the best model at ./results/checkpoint-1150/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=1250, training_loss=7.467929867553711, metrics={'train_runtime': 3406.7533, 'train_samples_per_second': 169.516, 'train_steps_per_second': 2.648, 'total_flos': 8.373251808405504e+16, 'train_loss': 7.467929867553711, 'epoch': 1.3844875346260388})

In [ ]:
################# Model Save

In [ ]:
# Save the model to the specified directory
output_dir = "./mt5_finetuned_model_new"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")


Model and tokenizer saved to ./mt5_finetuned_model_new


In [ ]:
# Save to Google Drive
import shutil
shutil.move(output_dir, "/content/drive/MyDrive/mt5_finetuned_model")

print("Model saved to Google Drive!")


Model saved to Google Drive!


In [ ]:
#######################Evaluate

In [ ]:
# Evaluation dataset with options column
eval_dataset_new = eval_data["test"]
eval_dataset_new = eval_dataset_new.map(preprocess_function, batched=True)
columns_remove = ["news_article", "question_stem", "ans", "target_num", "sentences_containing_the_numeral_in_answer_options"]
eval_dataset_new = eval_dataset_new.remove_columns(columns_remove)
eval_dataset_new.set_format("torch")


Map:   0%|          | 0/14438 [00:00<?, ? examples/s]

In [ ]:
def evaluate_samples(dataset, model, tokenizer, max_length=2, max_retries=3):
    results = []
    model.eval()
    with torch.no_grad():
        for sample in dataset:
            # Retrieve and process input, target, and options
            input_ids = sample["input_ids"].unsqueeze(0).to(model.device)  # Add batch dimension
            labels = sample["labels"].unsqueeze(0).to(model.device)     # Target values
            target_text = tokenizer.decode(labels[0], skip_special_tokens=True).strip()  # Decode target
            options = sample["answer_options"]  # Options provided in the dataset

            # Skip this sample if the target is not in the options
            if target_text not in options:
                continue

            predicted_text = ""
            retries = 0

            # Generate predictions and handle empty predictions
            while not predicted_text.strip() and retries < max_retries:
                outputs = model.generate(
                    input_ids=input_ids,
                    max_length=max_length,
                    early_stopping=False
                )
                predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
                retries += 1

            # Ensure the prediction is in the options
            if predicted_text not in options:
                # Find the closest match among the options
                corrected_prediction = min(
                    options, key=lambda opt: len(set(opt) ^ set(predicted_text))
                )
            else:
                corrected_prediction = predicted_text

            # Append results if prediction is valid
            if corrected_prediction:
                results.append({
                    "input": tokenizer.decode(input_ids[0], skip_special_tokens=True),
                    "target": target_text,
                    "prediction": predicted_text,
                    "corrected_prediction": corrected_prediction,
                })

    return results


In [ ]:
def calculate_accuracy(results):
    correct = 0
    total = len(results)

    for result in results:
        # Compare whether target and prediction are equal
        if result["target"] == result["prediction"]:
            correct += 1

    # Ensure total is not 0
    accuracy = correct / total if total > 0 else 0.0
    return accuracy


In [ ]:
# from torch.utils.data import Subset

#
# subset_indices = list(range(50))  # Select the first 50 data
# subset_eval_dataset = Subset(eval_dataset_new, subset_indices)


In [ ]:
# Evaluate
results = evaluate_samples(eval_dataset_new, model, tokenizer)
# Compute Accuracy
accuracy = calculate_accuracy(results)
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 87.00%


In [ ]:
#################

In [ ]:
# Check
for i, result in enumerate(results[:50]):
    print(f"Sample {i + 1}:")
    print(f"Input: {result['input']}")
    print(f"Target: {result['target']}")
    print(f"Prediction: {result['prediction']}")
    print("-" * 50)

Sample 1:
Input: Choose a correct answer to the following questions. Context: [['而是前一世代的iPhone 6s', '除iPhone 6s', 'iPhone 6s Plus分居第3', 'Galaxy S7/S7 Edge去年全球出貨量較前一代機種(Galaxy S6/S6 Edge)多出1,000萬支'], ['BBK/VIVO的6.0%'], ['2016年全球最暢銷(出貨量最多)的智慧手機產品不是蘋果(Apple)去年9月推出的iPhone 7', 'iPhone 7排第2', 'iPhone 7之外', 'iPhone 7 Plus', '全球最暢銷機種為iPhone 7', 'iPhone 7 Plus排第2', '旗艦機種Galaxy Note 7雖因連環爆而召回停售', '其中Galaxy S7 Edge', 'Galaxy S7分居第5', 'Galaxy S7/S7 Edge去年全球出貨量較前一代機種(Galaxy S6/S6 Edge)多出1,000萬支', 'OPPO推出的OPPO A53位居第7位', '且表現優於三星S7', '市佔排名從2015年的第7位躍升至第4位'], ['OPPO的7.2%']] Question: iPhone 6s為全球最暢銷智慧機;OPPO A53勝過三星S___ Options: ['6', '6.0', '7', '7.2'] Answer:
Target: 7
Prediction: 7
--------------------------------------------------
Sample 2:
Input: Choose a correct answer to the following questions. Context: [['前2月合併營收年增15%', '而Q2在NB', 'Q2營收仍可維持年增表現', '建準今年擬配息2元', '2月雖有過年與工作天數較少等因素干擾', '2月單月都呈年增表現', '累計前2月合併營收達15.8億元', '建準3月營收將較2月明顯彈升', '展望Q2', '建準Q2營收將可較Q1走揚'], ['2016年EPS為2.28元'], ['3月隨著工作天數回升', '